# EE4309 Object Detectation Project - Google Colab Version

This notebook maintains the same experience as the local Makefile commands, allowing you to easily run the project on Colab.

## 📝 Important Notes
1. **Enable GPU**: Click `Runtime` → `Change runtime type` → Select `T4 GPU`
2. **Run Order**: Please run cells in sequence
3. **Command Mapping**: Each cell corresponds to a make command

## 🔧 Step 0: Clone Project and Enter Directory

In [ ]:
# !git clone https://github.com/Enderfga/EE4309_proj.git
%cd EE4309_proj

c:\Users\Admin\Desktop\NUS\Y4S1\EE4309\Labs\EE4309_proj
c:\Users\Admin\Desktop\NUS\Y4S1\EE4309\Labs\EE4309_proj


In [ ]:
%matplotlib inline

## 📦 Step 1: Setup (equivalent to `make setup`)

In [2]:
# Install project dependencies (equivalent to make setup)
!pip install -q -r requirements.txt -e .

In [3]:
# Verify environment
import torch
print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

✅ PyTorch version: 2.8.0+cpu
✅ CUDA available: False


## 📋 Help (equivalent to `make help`)

In [8]:
# Show all available commands (equivalent to make help)
!make help

/usr/bin/bash: Files\Git\bin\bash.exe: No such file or directory


## 🖼️ Step 2: Download Samples (equivalent to `make samples`)

In [ ]:
# Download sample images (equivalent to make samples)
!make samples

## 🚀 Step 3: Train Model (equivalent to `make train`)

**Note**: On Colab, consider using fewer epochs to save time

## Modifications made to training

1. Add ColourJitter to training transforms
2. Change optimiser to AdamW and scheduler to CosineAnnealingLR
3. Implement gradient accumulation
4. Train with larger batch size (we run into CUDA memory limits using any higher batch sizes)
5. Plot training and validation losses across epochs
6. t-SNE dimensionality reduction visualisation

In [ ]:
# Standard training (equivalent to make train)
# model choices: "reset50", "vit"
!make train BATCH_SIZE=1 EPOCHS=2 MODEL="vit"

## 📊 Step 4: Evaluate Model (equivalent to `make eval`)

In [ ]:
# Evaluate the model (equivalent to make eval)
!make eval

## 🔍 Step 5: Run Inference (equivalent to `make infer`)

In [ ]:
# Run inference (equivalent to make infer)
!make infer

## 📝 Submit Your Work (equivalent to `make submit`)

**Important**: This will create a git commit with your student information

In [ ]:
# Submit your work with student information (equivalent to make submit)
# You will be prompted to enter:
# - Your full name
# - Your student ID (format: A0123456X)
# - Optional additional message
# Please run this command before compressing and submitting your project.
!make submit

---

## 💾 Extra: Save to Google Drive

In [ ]:
# Mount Google Drive to save training results
from google.colab import drive
drive.mount('/content/drive')

# Create save directory
!mkdir -p /content/drive/MyDrive/EE4309_results

# Copy training results
!cp -r runs/* /content/drive/MyDrive/EE4309_results/
print("✅ Results saved to Google Drive!")

## 📈 Visualize Inference Results

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

# Detection result images to display
detection_images = [
    'runs/infer_vis/dog_det.jpg',
    'runs/infer_vis/eagle_det.jpg', 
    'runs/infer_vis/horses_det.jpg',
    'runs/infer_vis/person_det.jpg'
]

# Check which images exist
existing_images = [img for img in detection_images if os.path.exists(img)]

if len(existing_images) >= 4:
    print("Displaying inference detection results:")
    
    # Create 2x2 grid
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Object Detection Results', fontsize=16, fontweight='bold')
    
    # Flatten axes for easy iteration
    axes_flat = axes.flatten()
    
    for i, img_path in enumerate(existing_images[:4]):
        try:
            img = mpimg.imread(img_path)
            axes_flat[i].imshow(img)
            # Extract image name without extension for title
            img_name = os.path.basename(img_path).replace('_det.jpg', '')
            axes_flat[i].set_title(img_name.title(), fontsize=14)
            axes_flat[i].axis('off')
        except Exception as e:
            axes_flat[i].text(0.5, 0.5, f'Error loading\n{os.path.basename(img_path)}', 
                            ha='center', va='center', transform=axes_flat[i].transAxes)
            axes_flat[i].axis('off')
    
    plt.tight_layout()
    plt.show()
    
elif len(existing_images) > 0:
    print(f"Found {len(existing_images)} detection results:")
    
    # Calculate grid size for available images
    n_images = len(existing_images)
    n_cols = min(2, n_images)
    n_rows = (n_images + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(8*n_cols, 6*n_rows))
    fig.suptitle('Available Detection Results', fontsize=16, fontweight='bold')
    
    # Handle single image case
    if n_images == 1:
        axes = [axes]
    elif n_rows == 1:
        axes = axes if hasattr(axes, '__iter__') else [axes]
    else:
        axes = axes.flatten()
    
    for i, img_path in enumerate(existing_images):
        try:
            img = mpimg.imread(img_path)
            axes[i].imshow(img)
            img_name = os.path.basename(img_path).replace('_det.jpg', '')
            axes[i].set_title(img_name.title(), fontsize=14)
            axes[i].axis('off')
        except Exception as e:
            axes[i].text(0.5, 0.5, f'Error loading\n{os.path.basename(img_path)}', 
                        ha='center', va='center', transform=axes[i].transAxes)
            axes[i].axis('off')
    
    # Hide empty subplots
    for i in range(n_images, len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()
    
else:
    print("❌ No detection result images found.")
    print("Please run inference first:")
    print("!make infer")

## 📊 Check Git History (View Submission Records)

In [ ]:
# View git commit history to check submissions
!git log --oneline -n 5

In [ ]:
# View detailed changes in the last commit
!git diff HEAD~1 --stat

## ⚠️ Troubleshooting

### 1. GPU Out of Memory
```bash
# Reduce batch size
!bash scripts/train.sh --batch-size 64 --epochs 20
```

### 2. Runtime Limits
- Colab free tier has ~12 hour limit
- Use checkpoint feature to train in segments
- Save to Google Drive regularly

### 3. Disconnection Issues
- Use resume training feature above to continue
- Consider Colab Pro for longer runtime

### 4. Submission Issues
- Make sure you run `make submit` before the deadline
- Use `git log` to verify your submission was recorded
- Your instructor can use `git diff` to review your changes